In [1]:
""" 
The algorithm is tested on the PDSystemEnv  gym task 
and developed with Tensorflow

Author: Daniel Salgado Rojo
"""
import tensorflow as tf
import numpy as np
import gym
import gym_pdsystem
from gym import wrappers
#import tflearn
import argparse
import pprint as pp

#from ddpg.replay_buffer import ReplayBuffer

from datetime import datetime

import random
from gym_pdsystem.envs.pdsystem_env import PDSystemEnv

/home/dsalgador/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
episode_length = 30
seed = 42

system = PDSystemEnv()
env = gym.make("PDSystemEnv-v0")
env._max_episode_steps = episode_length

np.random.seed(seed)
tf.set_random_seed(seed)
env.seed(seed)

state = env.reset()
state

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


array([ 35.58499034, 171.27643223,  67.48749111, 429.96423565,
        42.43103839])

In [3]:
a = np.array([1,0,2,0,5])
np.argmin(a)

1

In [4]:
min_boolean_indexes = np.isin(a, np.min(a))
min_indexes = np.where(min_boolean_indexes)
min_indexes[0]

array([1, 3])

In [5]:
def emptiest_tank_policy(s, system):
    min_boolean_indexes = np.isin(s, np.min(s))
    min_indexes = np.where(min_boolean_indexes)
    min_indexes = min_indexes[0]
    #n_min = len(min_indexes)

    possible_indexes = []
    
    for tank_visited in min_indexes:
        hypothetical_next_tank_state = state[tank_visited] + system.truck_max_loads
        print(hypothetical_next_tank_state, system.tank_max_loads[tank_visited])
        if hypothetical_next_tank_state <= system.tank_max_loads[tank_visited]:
                    possible_indexes.append(tank_visited)
    
    
    if len(possible_indexes) != 0:
        tank_to_go = random.choice(possible_indexes)
        return np.array([tank_to_go])
    else:
        return np.array([system.n]) #stay in the depot

In [6]:
a = emptiest_tank_policy(state, system)
a

[85.58499034] 100.0


array([0])

In [7]:
state2, r, terminal, info = env.step(a)
state2

action 0


array([ 69.08499034, 167.27643223,  56.98749111, 345.96423565,
         8.43103839])

In [8]:
system.tank_max_loads

array([100., 200., 100., 800., 200.])